In [30]:
import os

import numpy as np

In [31]:
# Tetraloop stats

tloop_seqs = np.load('tloop_seqs.npz')
clusters_dir = '../../../all_clusters'

all_tloop_files = [file for folder in os.listdir(clusters_dir) for file in os.listdir(f'{clusters_dir}/{folder}')]
print(f'No. of tetraloop .align.pdb files: {len(all_tloop_files)}')

tloops = [tloop for pdb_list in tloop_seqs.values() for tloop in pdb_list]
print(f'No. of unique tetraloops: {len(tloops)}')

tloop_pdb_ids = list(tloop_seqs.keys())
print(f'No. of tloop PDB IDs: {len(tloop_pdb_ids)}')

No. of tetraloop .align.pdb files: 19383
No. of unique tetraloops: 12027
No. of tloop PDB IDs: 864


In [32]:
# All sequence stats

all_seqs = np.load('all_seqs.npz')

pbd_ids = list(all_seqs.keys())
print(f'No. of PDB IDs: {len(pbd_ids)}')

# number of aligned tetraloops i.e. number of times a nonzero integer appears in the cluster_ids datarow

No. of PDB IDs: 864
